# 2_view_thalamus_celltypes_ccf

This notebook demonstrates how to use the custom plotting functions found in the
`abc_merfish_analysis\ccf_plots.py` module to view the thalamus subset of 
the Allen Brain Cell (ABC) Atlas's whole mouse brain MERFISH dataset 
(https://portal.brain-map.org/atlases-and-data/bkp/abc-atlas), which we learned 
how to load in `1_load_thalamus_data notebook.ipynb`.

In [1]:
import numpy as np
import pandas as pd

from abc_merfish_analysis import abc_load as abc
from abc_merfish_analysis import ccf_images as cimg
from abc_merfish_analysis import ccf_plots as cplots
from colorcet import glasbey

get_ipython().run_line_magic('matplotlib', 'inline') 

## Load thalamus dataset

Since we're only going to view cell types & CCF structures in this notebook, 
we can just load the cell metadata DataFrame. For details on the DataFrame output 
of `load_standard_thalamus()`, please see `1_load_thalamus_data.ipynb`.

For gene expression plots, see notebook `3_view_thalamus_gene_expression.ipynb`.

In [2]:
# load cell metadata DataFrame
obs_th = abc.load_standard_thalamus(data_structure='obs')

### Set spatial coordinates

Since the ABC Atlas MERFISH data includes 3 sets of spatial coordinates, 
(`_section`, `_reconstructed`, and `_ccf`), all stored in `.obs,
we need to specify which coordinate space we wish to work from.
```
x_col, y_col : str
    Column names in obs for the x and y coordinates of cells
```

A column to use for section labels also needs to be specified, defaulting
to 'brain_section_label' as in the web atlas. A z-coordinate in the section
coordinate system also works 

(z_section and z_reconstructed are identical, while other
z-coordinates vary slightly across the section and thus can't be used here).
```
section_col : str, {'brain_section_label', 'z_section', 'z_reconstructed'}
    Column name in obs for the section labels in 'sections'
```

For this notebook, we will work in the `_reconstructed` coordinate space, which is a registration of each MERFISH section to a target resampled CCF section. 
For more information on the different coordinates, please see: https://alleninstitute.github.io/abc_atlas_access/notebooks/merfish_ccf_registration_tutorial.html

In [3]:
# show the three sets of coordinates
display(obs_th[['x_section', 'y_section', 'z_section',
                'x_reconstructed', 'y_reconstructed', 'z_reconstructed',
                'x_ccf', 'y_ccf', 'z_ccf', 'brain_section_label']
              ].head())

In [4]:
# set spatial column names to `_reconstructed` coordinate space
coords = '_reconstructed'

x_col = 'x'+coords
y_col = 'y'+coords
section_col = z_col = 'z'+coords

### Load Allen Reference Atlas (ARA) structure images & boundaries

To overlay the ARA structures on the cell types found in the thalamus, we need to load the volume & boundary images from the ABC Atlas, which are referred to as 'CCF' from here on out.

In [5]:
# Load the CCF structures image
ccf_images = abc.get_ccf_labels_image()

In [6]:
# Precompute boundaries to use multiple times in this notebook w/o need to recompute

# we only need to load the boundaries for sections that contain the thalamus
sections_all = sorted(obs_th[section_col].unique())
sections_int = np.rint(np.array(sections_all)/0.2).astype(int)

# precompute boundaries for thalamic sections
ccf_boundaries = cimg.sectionwise_label_erosion(ccf_images, distance_px=1, 
                                                fill_val=0, return_edges=True, 
                                                section_list=sections_int
                                                )

# # Alternative: skip and define as None to calculate on the fly
# ccf_boundaries = None

## Set default figure behavior

### Select sections to display

The plotting code in ccf_plots.py module displays figures per-sections, so you may wish to specify which section(s) to display.

`abc.TH_EXAMPLE_Z_SECTIONS` stores 3 sections that are representative along AP axis of the thalamus and can be used to speed up the run time of this notebook by setting `sections=abc.TH_EXAMPLE_Z_SECTIONS` in all the plotting functions.

If you wish to see all sections, you can: set the `sections=` parameter to `None`, not explicitly set it (`default=None`), or set it to `sections=sections_all`

In [7]:
# 3 sections that representatively span the AP axis of the thalamus
abc.TH_EXAMPLE_Z_SECTIONS

### Set figure saving behavior

In [8]:
save_figs = False

### Set default CCF regions to display

In [9]:
# if you reload abc_merfish_analysis.ccf_plots after this cell has been run, 
# cplots.CCF_REGIONS_DEFAULT will be reset to None & you'll need to re-run this cell
cplots.CCF_REGIONS_DEFAULT = abc.get_thalamus_names()

## View thalamus cell types + CCF parcellation

We'll use the custom plotting functions in the `abc_merfish_analysis/ccf_plot.py`, aka `cplots`, module to view the cell types and the aligned CCF parcellation structures

### View just CCF structures (no cells yet!)

In [10]:
# Display CCF structures with face colored by random palette
ccf_shape_figs = [cplots.plot_ccf_section(ccf_images, sec,
                                           boundary_img=ccf_boundaries, 
                                           section_col=section_col,
                                           face_palette=glasbey, # set CCF structure face color palette
                                           edge_color='black', # set CCF structure boundary color
                                           legend=True)
                    for sec in abc.TH_EXAMPLE_Z_SECTIONS]
# save as pdfs
if save_figs:
    for i, fig in enumerate(ccf_shape_figs):
        sec = int(abc.TH_EXAMPLE_Z_SECTIONS[i]*10)
        fig.savefig(f'/results/ccf_shape_z{sec}.pdf', transparent=True, 
                    bbox_inches='tight')

In [11]:
# diplay just the outlines of the CCF structures, no face color or legend
ccf_outline_figs = [cplots.plot_ccf_section(ccf_images, sec,
                                           boundary_img=ccf_boundaries, 
                                           section_col=section_col,
                                           legend=False)
                    for sec in abc.TH_EXAMPLE_Z_SECTIONS]

### View cell types + CCF outline overlay

##### Load cell type color palettes

We provide two ways of selecting pre-set color palette to visualize cell types:
1. Use the published color palettes for the ABC Atlas taxonomy levels
2. Use a custom cluster color palette to increase contrast between spatially neighboring clusters in the thalamus

In [12]:
# We can load & use the published color palettes for the ABC Atlas taxonomy
abc_palettes = {level: abc.get_taxonomy_palette(level) for level in 
                ['neurotransmitter','class', 'subclass','supertype','cluster']}

# We have also provided a secondary color palette for the cluster level that 
# attempts to increase the color contrast between spatially negihboring clusters
cluster_palette = abc.get_thalamus_cluster_palette()

#### Set plot kwargs

So we don't have to set these for each individual plot function call

In [13]:
kwargs_celltypes = dict(boundary_img=ccf_boundaries,
                        section_col=section_col,
                        x_col=x_col, 
                        y_col=y_col,
                        point_size=2, # cell marker size
                       )

#### View cells colored by neurotransmitter

In [14]:
taxonomy_level = 'neurotransmitter'
nt_figs = cplots.plot_ccf_overlay(obs_th, ccf_images, 
                                        point_hue=taxonomy_level, 
                                        sections=abc.TH_EXAMPLE_Z_SECTIONS,
                                        point_palette=abc_palettes[taxonomy_level],
                                        legend='cells',
                                        separate_figs=False,
                                        figsize=(20,3),
                                        **kwargs_celltypes)

if save_figs: 
    for i, fig in enumerate(nt_figs):
        sec = int(abc.TH_EXAMPLE_Z_SECTIONS[i]*10)
        fig.savefig(f'/results/ccf_{taxonomy_level}_z{sec}.pdf', transparent=True, 
                    bbox_inches='tight')

#### View cells colored by class

In [15]:
taxonomy_level = 'class'
class_figs = cplots.plot_ccf_overlay(obs_th, ccf_images, 
                                     point_hue=taxonomy_level, 
                                     sections=abc.TH_EXAMPLE_Z_SECTIONS,
                                     point_palette=abc_palettes[taxonomy_level],
                                     legend='cells',
                                     separate_figs=False,
                                     figsize=(20,3),
                                     **kwargs_celltypes)

if save_figs: 
    for i, fig in enumerate(class_figs):
        sec = int(abc.TH_EXAMPLE_Z_SECTIONS[i]*10)
        fig.savefig(f'/results/ccf_{taxonomy_level}_z{sec}.pdf', transparent=True, 
                    bbox_inches='tight')

#### View cells colored by subclass

##### View all thalamus sections

By setting `sections=None`, we can display all the sections in this thalamus dataset. We don't display a legend here to save space.

In [16]:
subclass_figs = cplots.plot_ccf_overlay(obs_th, ccf_images, 
                                        point_hue='subclass', 
                                        sections=None,
                                        point_palette=abc_palettes['subclass'],
                                        legend=None,
                                        separate_figs=False, n_rows=4,
                                        figsize=(20,20),
                                        **kwargs_celltypes)

if save_figs: 
    for i, fig in enumerate(subclass_figs):
        sec = int(abc.TH_EXAMPLE_Z_SECTIONS[i]*10)
        fig.savefig(f'/results/ccf_subclass_z{sec}.pdf', transparent=True, 
                    bbox_inches='tight')

##### View only 3 sample sections

There are few enough subclasses that it makes sense to display a legend using `legend='cells'`.

In [17]:
subclass_figs = cplots.plot_ccf_overlay(obs_th, ccf_images, 
                                        point_hue='subclass', 
                                        sections=abc.TH_EXAMPLE_Z_SECTIONS,
                                        point_palette=abc_palettes['subclass'],
                                        legend='cells',
                                        separate_figs=False,
                                        figsize=(20,3),
                                        **kwargs_celltypes)

if save_figs: 
    for i, fig in enumerate(subclass_figs):
        sec = int(abc.TH_EXAMPLE_Z_SECTIONS[i]*10)
        fig.savefig(f'/results/ccf_subclass_z{sec}.pdf', transparent=True, 
                    bbox_inches='tight')

#### View cells colored by supertype

There are still few enough supertypes that we'll keep the legend.

In [18]:
taxonomy_level = 'supertype'
supertype_figs = cplots.plot_ccf_overlay(obs_th, ccf_images, 
                                        point_hue=taxonomy_level, 
                                        sections=abc.TH_EXAMPLE_Z_SECTIONS,
                                        point_palette=abc_palettes[taxonomy_level],
                                        legend='cells',
                                        separate_figs=False,
                                        figsize=(20,3),
                                        **kwargs_celltypes)

if save_figs: 
    for i, fig in enumerate(supertype_figs):
        sec = int(abc.TH_EXAMPLE_Z_SECTIONS[i]*10)
        fig.savefig(f'/results/ccf_{taxonomy_level}_z{sec}.pdf', transparent=True, 
                    bbox_inches='tight')

#### View cells colored by cluster

There are too many clusters for the legend to be readable, so we'll turn it off for this plot by setting `legend=None`.

We will also use the custom color palette loaded above to increase the contrast of neighboring clusters.

In [19]:
taxonomy_level = 'cluster'
cluster_figs = cplots.plot_ccf_overlay(obs_th, ccf_images, 
                                        point_hue=taxonomy_level, 
                                        sections=abc.TH_EXAMPLE_Z_SECTIONS,
                                        point_palette=cluster_palette,
                                        legend=None,
                                        separate_figs=False,
                                        figsize=(20,3),
                                        **kwargs_celltypes)

if save_figs: 
    for i, fig in enumerate(cluster_figs):
        sec = int(abc.TH_EXAMPLE_Z_SECTIONS[i]*10)
        fig.savefig(f'/results/ccf_{taxonomy_level}_z{sec}.pdf', transparent=True, 
                    bbox_inches='tight')

You can compare this custom color palette to the published color palette for clusters:

In [20]:
taxonomy_level = 'cluster'
cluster_figs = cplots.plot_ccf_overlay(obs_th, ccf_images, 
                                        point_hue=taxonomy_level, 
                                        sections=abc.TH_EXAMPLE_Z_SECTIONS,
                                        point_palette=abc_palettes[taxonomy_level],
                                        legend=None,
                                        separate_figs=False,
                                        figsize=(20,3),
                                        **kwargs_celltypes)


## View cluster annotations for each nucleus

In [21]:
# load manual cluster annotations for thalamic nuclei
nuclei_cluster_df = abc.nuclei_df_manual
nuclei_cluster_df.head(5)

In [22]:
kwargs_cluster_annotations = dict(
    bg_cells=obs_th,
    boundary_img=ccf_boundaries,
    section_col=section_col,
    x_col=x_col,
    y_col=y_col,
    point_size=3, 
    face_palette=None,
    edge_color='silver'
    )

In [23]:
# nucleus of interest
nucleus = 'MD'

# Let's look at the cell types in the MD in one sample section
# sections_VM = [7.2]
nuclei_highlight = [nucleus]
taxonomy_level = 'cluster'

# get the cluster annotations for the nucleus of interest
obs_annot = abc.get_obs_from_annotations([nucleus], obs_th, taxonomy_level=taxonomy_level)

figs_annot = cplots.plot_ccf_overlay(obs_annot, 
                                    ccf_images, 
                                    ccf_highlight=nuclei_highlight,
                                    point_hue=taxonomy_level, 
                                    sections=None,
                                    point_palette=cluster_palette,
                                    legend='cells',
                                    separate_figs=False, n_rows=3,
                                    figsize=(20,10),
                                    **kwargs_cluster_annotations)

In [24]:
# get the cluster annotations for the nucleus of interest
nucleus = ['VM']

# Let's look at the cell types in the ATN in one sample section
# sections_VM = [7.2]
nuclei_highlight = nucleus
taxonomy_level = 'cluster'

obs_annot = abc.get_obs_from_annotations(nucleus, obs_th, taxonomy_level=taxonomy_level)

figs_annot = cplots.plot_ccf_overlay(obs_annot, 
                                    ccf_images, 
                                    ccf_highlight=nuclei_highlight,
                                    point_hue=taxonomy_level, 
                                    sections=None,
                                    point_palette=cluster_palette,
                                    legend='cells',
                                    separate_figs=False, n_rows=3,
                                    figsize=(20,10),
                                    **kwargs_cluster_annotations)

In [25]:
# get the cluster annotations for the nucleus of interest
nucleus = ['VAL']

# Let's look at the cell types in the ATN in one sample section
# sections_VM = [7.2]
nuclei_highlight = nucleus
taxonomy_level = 'cluster'

obs_annot = abc.get_obs_from_annotations(nucleus, obs_th, taxonomy_level=taxonomy_level)

figs_annot = cplots.plot_ccf_overlay(obs_annot, 
                                    ccf_images, 
                                    ccf_highlight=nuclei_highlight,
                                    point_hue=taxonomy_level, 
                                    sections=None,
                                    point_palette=cluster_palette,
                                    legend='cells',
                                    separate_figs=False, n_rows=3,
                                    figsize=(20,10),
                                    **kwargs_cluster_annotations)

In [26]:
# get the cluster annotations for the nucleus of interest
nucleus = ['AD', 'AM', 'AV']

# Let's look at the cell types in the ATN in one sample section
nuclei_highlight = ['AD', 'AMd', 'AMv', 'AV', 'AV']
taxonomy_level = 'cluster'

obs_annot = abc.get_obs_from_annotations(nucleus, obs_th, taxonomy_level=taxonomy_level)

figs_annot = cplots.plot_ccf_overlay(obs_annot, 
                                    ccf_images, 
                                    ccf_highlight=nuclei_highlight,
                                    point_hue=taxonomy_level, 
                                    sections=None,
                                    point_palette=cluster_palette,
                                    legend='cells',
                                    separate_figs=False, n_rows=2,
                                    figsize=(20,10),
                                    **kwargs_cluster_annotations)

In [30]:
# get the cluster annotations for the nucleus of interest
nucleus = 'PVT'

# Let's look at the cell types in the ATN in one sample section
# sections_VM = [7.2]
nuclei_highlight = [nucleus]
taxonomy_level = 'cluster'

obs_annot = abc.get_obs_from_annotations(nucleus, obs_th, taxonomy_level=taxonomy_level)

figs_annot = cplots.plot_ccf_overlay(obs_annot, 
                                    ccf_images, 
                                    ccf_highlight=nuclei_highlight,
                                    point_hue=taxonomy_level, 
                                    sections=None,
                                    point_palette=cluster_palette,
                                    legend='cells',
                                    separate_figs=False, n_rows=3,
                                    figsize=(20,10),
                                    **kwargs_cluster_annotations)